# VESIcal at vVMSG 2022
Simon Matthews (University of Iceland) & Penny Wieser (Oregon State University)

## 03 - Batch Calculations and Isobars
So far you have seen how calculations can be run for individual sample compositions. You could use a loop to run through a whole dataset of melt inclusion compositions, but VESIcal has this capability baked in already. 

In this notebook we will demonstrate how to do these calculations, and how to make isobar plots.

First we need to import the python modules we want to use:

In [ ]:
# NumPy provides tools for doing numerical calculations
import numpy as np

# VESIcal provides the tools for performing volatile solubility calculations
import VESIcal as v

# Pandas provides tools for working with large data tables
import pandas as pd

# Matplotlib provides tools for plotting data and model results
import matplotlib.pyplot as plt

### Exercise 03.01 - Read in a dataset
For the purposes of the workshop we will use a melt inclusion dataset from Cassidy et al. (2015), though you could easily substitute your own. Just make sure you have properly named column headings (i.e., like those in the example file).

The file is read in to VESIcal as a `BatchFile`:

In [ ]:
myfile= v.BatchFile('Cassidy2015_AndesiteMI_YouTubeDemo.xlsx', 
                    sheet_name='Sheet1',  
                    input_type='wtpercent')

We can look at this data within the notebook, which also allows us to check that it has been imported correctly:

In [ ]:
myfile.get_data()

### Exercise 03.02 - Calculate saturation pressures using MagmaSat
Doing the batch calculations is just as straightforward as the individual samples. As before the default option is MagmaSat. 

The sample composition and temperature to use is provided by the table, but we need to tell VESIcal what the temperature column is named. The calculation may take a little while:

In [ ]:
SatP_MagmaSat= myfile.calculate_saturation_pressure(temperature="Temp")

We can view the results that have been stored in `SatP_MagmaSat`. You need to scroll right to find the saturation pressure and related information.

In [ ]:
SatP_MagmaSat

We can also save the output to an excel file, or a csv file:

In [ ]:
SatP_MagmaSat.to_excel('SatP_Cassidy_MagmaSat.xlsx')

But we can also do some data analysis right here. For a start let's plot the saturation pressures:

In [ ]:
# Tell python we are creating a figure, containing an axis
fig, ax = plt.subplots(dpi=100)

# Make a scatter plot of SiO2 vs Saturation P
ax.scatter(SatP_MagmaSat['SiO2'], SatP_MagmaSat['SaturationP_bars_VESIcal'])

# Label the axes:
ax.set_xlabel("SiO$_2$ (wt%)")
ax.set_ylabel("Saturation\nPressure (bars)")

# Show the figure
plt.show()

We could save this figure using a variety of formats, but pdf is convenient:

In [ ]:
fig.savefig("MagmaSatP.pdf")

### Exercise 03.03 - Plotting
Try plotting the calculated saturation pressures against a different chemical parameter. Either modify the code above, or copy and paste it into a new cell below.

### Exercise 03.04 - Compare results from different models

We can now repeat the calculation using another model, the Dixon model for example:

In [ ]:
SatP_Dixon= myfile.calculate_saturation_pressure(temperature="Temp",
                                                 model='Dixon')

Now we can plot the results against each other:

In [ ]:
fig, ax = plt.subplots(dpi=100)

ax.scatter(SatP_MagmaSat['SaturationP_bars_VESIcal'],
           SatP_Dixon['SaturationP_bars_VESIcal'])

# Plot a 1:1 line, but getting the axis limits, locking them in,
# and then plotting a line between them
ax.set_xlim(ax.get_xlim())
# ax.set_ylim(ax.get_ylim())
ax.plot(ax.get_xlim(), ax.get_xlim(), c='black')

# Set axis labels
ax.set_xlabel("MagmaSat (bars)")
ax.set_ylabel("Dixon (bars)")

plt.show()

Where is this discrepancy coming from? Try making a plot of the discrepancy (ratio) between the saturation pressures and the SiO$_2$ content.

If you have time, try repeating the comparison with a different model.

### Exercise 03.05 - Calculate isobars
Isobars are contours in CO$_2$ vs H$_2$O plots of constant pressure. VESIcal has a routine to calculate these automatically.

We will do this for a single composition as it takes a while to calculate them. We can extract a single sample from the dataset we imported earlier, if we know the sample name:

In [ ]:
sample = myfile.get_sample_composition('SSH4_1', asSampleClass=True)

*Aside- in Jupyter notebooks you can view the documentation for different methods and functions by writing a ? after their name and running the cell. You can see all the different arguments you can use when calling the method, and a description of what it does*

In [ ]:
myfile.get_sample_composition?

Let's do the isobars calculation, first we have to set up how we want the calculation to be run. We will also calculate isopleths (contours of constant fluid compositions):

In [ ]:
T = 1200 # degrees C
pressures = [1000, 2000, 3000] # bars
isopleths = [0, 0.25, 0.5, 0.75, 1.0] # Mole fractions

In [ ]:
calc = v.calculate_isobars_and_isopleths(sample=sample, 
#                                          smooth_isobars=False,
                                         temperature=T,
                                         pressure_list=pressures,
                                         isopleth_list=isopleths,
                                         print_status=True)

Extract the calculated isobars and isopleths:

In [ ]:
isobars, isopleths = calc.result

The results are stored in a table, which we can look at:

In [ ]:
isobars

Now we can plot the results:

In [ ]:
fig, ax = plt.subplots()

for p in isobars['Pressure'].unique():
    ax.plot(isobars[isobars['Pressure']==p].H2O_liq,
            isobars[isobars['Pressure']==p].CO2_liq,
            label='{:.0f}'.format(p))

for i in isopleths['XH2O_fl'].unique():
    ax.plot(isopleths[isopleths['XH2O_fl']==i].H2O_liq,
            isopleths[isopleths['XH2O_fl']==i].CO2_liq,
            c='black')
    
ax.scatter(myfile.get_data()['H2O'],
           myfile.get_data()['CO2'],
           label='Melt Inclusions')

ax.legend()

ax.set_xlabel('H$_2$O (wt%)')
ax.set_ylabel('CO$_2$ (wt%)')

plt.show()

But is this a reliable way of plotting data? Penny will explain (and demonstrate with a time consuming calculation!)...